In [ ]:
import pandas as pd
import plotly.express as px
import seaborn as sns

In [ ]:
df = pd.read_csv("C:/Users/danie/Downloads/SmA-Four-Tank-Batch-Process_V2.csv", sep = ";")

In [ ]:
col = []

# Entferne redundante Bezeichnungen aus den Spaltennamen
for x in df.columns:
    col.append(x.split(" ")[0])
    
df.columns = col

In [ ]:
# Entferne alle Schritte mit StepID = 2, da diese für das Projekt nicht relevant sind
df.drop(df[df["CuStepNo"] == 2].index, inplace = True)

In [ ]:
# Extrahiere die Zeitangaben bzw. formatiere die Zeitstempel in das richtige Format
df.timestamp = pd.to_datetime(df.timestamp)


df["day"] = df.timestamp.dt.day
df["hour"] = df.timestamp.dt.hour
df["second"] = df.timestamp.dt.second

In [ ]:
# Keine NaN-Werte vorhanden
df[df.any(axis = 1).isna()]

In [ ]:
# determine start and end of steps
df['dstep_p']=df['CuStepNo'].diff()
df['dstep_n']=df['CuStepNo'].diff(-1)

vsteps = [1,7,8,3]

# select rows with a step change
dfsen=df[(df['dstep_n']!=0)]
dfsep=df[(df['dstep_p']!=0)]
dfse=pd.concat([dfsen,dfsep])
dfse=dfse.sort_values(by=['timestamp'])

# create new dataframe where we store extracted information
dfinfo_steps=pd.DataFrame(columns=['step_length','start','end','stepn'])

# iterative approach
pstep=-1
c=0
for n in range(dfse.shape[0]):
    # get row
    r=dfse.iloc[n]
    if pstep==r['CuStepNo']:
        # determine step length
        stepl=r['timestamp']-dfse.iloc[n-1]['timestamp']
        # update dataframe
        dfinfo_steps.loc[c]=(stepl,dfse.iloc[n-1]['timestamp'],r['timestamp'],r['CuStepNo'])
        c=c+1
    else:
        pstep=r['CuStepNo']
print(dfinfo_steps.head())
print('Max step_length: {}'.format(dfinfo_steps['step_length'].max()))
print('Min step_length: {}'.format(dfinfo_steps['step_length'].min()))
print('#steps: {}'.format(dfinfo_steps.shape[0]))

# now determine whether the batch is complete
batchn=1
batchi=-1
dfinfo_steps["batchn"]=0
dfinfo_steps["is_complete"]=False
dfinfo_batches=pd.DataFrame(columns=['batch_length','start','end','steps','batchn','is_complete'])
n=0
b=0
while True:
    if n+len(vsteps)>dfinfo_steps.shape[0]:
        # complete info at incomplete, last batch
        steps=[]
        for v in range(dfinfo_steps.shape[0]-n):
            dfinfo_steps.at[n+v,'batchn']=batchi
            dfinfo_steps.at[n+v,'is_complete']=False
            steps.append(dfinfo_steps.at[n+v,'stepn'])
        dfinfo_batches.loc[b]=[dfinfo_steps.at[n+v,'end']-dfinfo_steps.at[n,'start'],dfinfo_steps.at[n,'start'], \
                               dfinfo_steps.at[n+v,'end'],steps,batchi,False]
        b=b+1
        break
    # check if all steps of a batch are present and in correct order
    isCorrect=True
    for v in range(len(vsteps)):
        isCorrect=dfinfo_steps.loc[n+v,'stepn']==vsteps[v]
        if not isCorrect:
            break
    if isCorrect:
        steps=[]
        for v in range(len(vsteps)):
            dfinfo_steps.at[n+v,'batchn']=batchn
            dfinfo_steps.at[n+v,'is_complete']=True
            steps.append(dfinfo_steps.at[n+v,'stepn'])
        dfinfo_batches.loc[b]=[dfinfo_steps.at[n+v,'end']-dfinfo_steps.at[n,'start'],dfinfo_steps.at[n,'start'], \
                               dfinfo_steps.at[n+v,'end'],steps,batchn,True]
        n=n+len(vsteps)
        batchn=batchn+1
        b=b+1
    else:
        steps=[]
        for vc in range(v):
            dfinfo_steps.at[n+vc,'batchn']=batchi
            dfinfo_steps.at[n+vc,'is_complete']=False
            steps.append(dfinfo_steps.at[n+v,'stepn'])
        dfinfo_batches.loc[b]=[dfinfo_steps.at[n+vc,'end']-dfinfo_steps.at[n,'start'],dfinfo_steps.at[n,'start'], \
                               dfinfo_steps.at[n+vc,'end'],steps,batchi,False]
        batchi=batchi-1
        n=n+vc
        b=b+1
# save dfinfo_steps to file
dfinfo_steps.to_pickle('SmA-Four-Tank-Info-Steps.pkl')
dfinfo_batches.to_pickle('SmA-Four-Tank-Info-Batches.pkl')


In [ ]:
df_steps = pd.read_pickle("C:/Users/danie/SmA-Four-Tank-Info-Steps.pkl")
df_batches = pd.read_pickle("C:/Users/danie/SmA-Four-Tank-Info-Batches.pkl")

In [ ]:
# Entferne den letzten unvollständigen Batch aus dem Datensatz

df = df.drop(df[df.timestamp >= '2018-10-31 14:29:32'].index)

In [ ]:
df[df.DeviationID == 0]

In [ ]:
# Entferne einen Zeileneintrag, bei dem die DeviationID 0 ist

df = df.drop(df[df.DeviationID == 0].index)

# DevId
### 1 --> Pl1200 (zwischen PL12002 und PL12003)
### 2 --> PL1100 (zwischen YC10001 und YC14001)
### 3 --> YC21006
### 4 --> YC22006
### 5 --> YC23006
### 6 --> YS14005
### 7 --> Pl2150
### 8 --> YS14004
### 9 --> YS10004

Schritt 7 generell am kürzesten, Schritt 8 am längsten (evtl. wegen Pausen zwischen den Batches? An Wochenenden wurden jedoch keine Daten gespeichert)

In [ ]:
df.groupby("CuStepNo").agg({"second":"sum"})

In [ ]:
df_steps.groupby("stepn").agg({"step_length":"sum"})

#### Wochenenden am:
13.-14.

20.-21.

27.-28.

### 16. und 23. Oktober komplett fehlerfrei

In [ ]:
df[df.DeviationID > 1].day.unique()

In [ ]:
df_steps.sort_values("step_length", ascending = False)

In [ ]:
# Füge Batchnummern und Schrittlängen zum Dataframe hinzu
a = df.merge(df_steps, how = "left", left_on = "timestamp", right_on = "start")

In [ ]:
a[["stepn"]] = a[["stepn"]].fillna(method = "ffill")
a[["batchn"]] = a[["batchn"]].fillna(method = "ffill")
a["step_length"] = a["step_length"].dt.total_seconds()
a[["step_length"]] = a[["step_length"]].fillna(0)

In [ ]:
# redundante oder unnötige Spalten entfernen
a.drop(columns = ["start", "end", "is_complete", "stepn"], inplace = True)

## Ein Batchprozess benötigt hier zwischen 468 und 1997 Sekunden.

In [ ]:
a.groupby(["batchn"]).agg({"timestamp":"count", "DeviationID":"unique", "CuStepNo":"unique"}).sort_values("timestamp")

### Schritt 8 in Batch 9 äußerst kurz

In [ ]:
a[a.batchn == 9].drop_duplicates(subset = ["step_length"])

In [ ]:
a.groupby("DeviationID").agg({"batchn":"nunique"})

In [ ]:
a.groupby("day").agg({"batchn":"nunique"})

### Bei 9 Spalten sind die gemessenene Werte konstant

--> nicht nützlich zur Anomaliedetektion/ Klassifizierung, kann entfernt werden

In [ ]:
useless_columns = []

for x in a.columns:
    if a[x].describe()[2] == 0.0:
        print(x)
        useless_columns.append(x)
        print(f"unique values: {a[x].unique()}")
        
a.drop(columns = useless_columns, inplace = True)

Die Werte in PIC14007_SP weisen keine signifikant großen Veränderungen auf

In [ ]:
a.drop(["PIC14007_SP"], axis = 1, inplace = True)

In [ ]:
a["DeviationID"] = a.DeviationID.astype(float)

In [ ]:
# Erstelle neue Spalte timestamp_difference, die den Zeitunterschied seit dem vorherigen Datenpunkt in Sekunden anzeigt.

a["timestamp_difference"] = a.timestamp.diff().dt.total_seconds()

In [ ]:
# Setze den Wert der zuvor erstellten Spalte bei jedem neuen Batch zu Beginn auf 0

a.loc[a.timestamp_difference > 80, "timestamp_difference"] = 0
a["timestamp_difference"].iloc[0] = 0

In [ ]:
# Erstelle neue Spalte batch_duration, die pro Batch die gesamt verstrichene Zeit seit Batchbeginn anzeigt.


a["batch_duration"] = 0

for x in range(1, a.batchn.nunique() + 1):
    # Pro Batchnummer wird die Batchzeit berechnet (sonst bekommt man am Ende die Gesamtdauer aller Batches)
    a.loc[a.batchn == x, "batch_duration"] = a.loc[a.batchn == x, :].timestamp_difference.cumsum()

In [ ]:
for x in range(1, a.batchn.nunique() + 1):
    for i in a[a.batchn == x].CuStepNo.unique():
        a.loc[(a.batchn == x) & (a.CuStepNo == i), "step_length"] = a.loc[(a.batchn == x) & (a.CuStepNo == i), :].timestamp_difference.cumsum()

In [ ]:
figs = []

# Erstelle für jede unterschiedliche DeviationID ein Plot, das den Verlauf eines bestimmten Werts pro Batch anzeigt

def create_batch_comparisons(data, column):
    n = 0
    for x in sorted(data.DeviationID.unique()):
        fig = px.line(data[(a.batchn == n) & (data.DeviationID == x)],  x = "batch_duration", 
                      y = column, 
                      title = f"DeviationID = {x}"
                 )

        for n in range(batchn): 
            fig.add_scatter(x = data[(data.batchn == n) & (data.DeviationID == x)]["batch_duration"], 
                            y = data[(data.batchn == n) & (data.DeviationID == x)][column])
        figs.append(fig)
        # jedes Element in figs steht für ein Plot mit allen Batches, die die gleiche DeviationID aufweisen
        # Achtung! Die traces in der Farblegende sind: batchn des Batches + 1
    return figs

gg = create_batch_comparisons(a, "YC14001_MV")

In [ ]:
a.columns

In [ ]:
for e in range(0, len(gg)):
    gg[e].show()
    

In [ ]:
from plotly.subplots import make_subplots

fig_s = []

# Erstelle für jede unterschiedliche DeviationID ein Plot, das den Verlauf eines bestimmten Werts pro Batch anzeigt

def create_step_comparisons(data, column, CuStep):
    n = 0
    
    
    for x in sorted(a.DeviationID.unique()):
        fig = px.line(data[(data.batchn == n) & (data.DeviationID == x)],  x = "step_length", 
                      y = column, 
                      title = f"DeviationID = {x}, StepID: {CuStep}", hover_name = "batchn",
                 )
        for n in range(1, batchn + 1):
            fig.add_scatter(x = data[(data.batchn == n) & (data.DeviationID == x) & (data.CuStepNo == CuStep)]["step_length"], 
                            y = data[(data.batchn == n) & (data.DeviationID == x) & (data.CuStepNo == CuStep)][column],
                            )
        fig_s.append(fig)
        # jedes Element in figs steht für ein Plot mit allen Batches, die die gleiche DeviationID aufweisen
        # Achtung! Die traces in der Farblegende sind: batchn des Batches + 1
    return fig_s

gg_s = create_step_comparisons(a, "LevelMainTank", 1)

In [ ]:
for e in range(0, len(gg_s)):
    gg_s[e].show()
    

In [ ]:
dtw_df[(dtw_df.batchn == 1) & (dtw_df.DeviationID == 1) & (dtw_df.CuStepNo == 1) & (dtw_df.step_length > 25)]

## Überblick über alle gemessenen Werte

manche Fehlertypen evtl. nur durch Analysieren der zeitlichen Abstände erkennbar

##### LevelMainTank um die Hälfte geringer ab 24.10.?

##### PI12002_PV_Out ungefähr am 16.10. auf 0 ?

##### FIC14002_SP kurz 0 am 26./27.10. --> Neustart des Systems?

### DevID = 2.0 oder 3.0 
--> YC14001_MV, FIC14002_MV nur auf 100?

### DevID = 4.0 
--> FIC14002_PV_Out über 5000, 

FIC14002_MV unter 30, 

YC14006_MV sehr hoch, 

PI14012_PV_Out sehr hoch (auch bei DevID = 3.0)

PIC14007_MV sehr hoch


### DevID = 8.0 
--> YC14006_MV deutlich niedriger, 

PI14012_PV_Out unter 0 (beide Bauteile hängen miteinander zusammen), 

YC23001_MV auf 100, 

FIC23002_MV auf 100, 

FIC14002_SP minimal

In [ ]:
px.histogram(a.resample("5S", on = "timestamp").mean().reset_index(), x = "batchn", nbins = 300,
            title = "Wurden alle Batches gleich oft im Datensatz gespeichert?", color = "DeviationID", width = 1000)

In [ ]:
days = [a[a["PI12002_PV_Out"] < 0].day.unique()[0]]

fig_days = px.line(a[(a.day.isin(days)) & (a.timestamp.dt.hour < 18)], x = "timestamp", y = "PI12002_PV_Out")

In [ ]:
for r in a[(a.day.isin(days)) & (a.timestamp.dt.hour < 15)].drop_duplicates(["batchn", "CuStepNo"]).iloc:
    fig_days.add_vline(x = r.timestamp)
    fig_days.add_annotation(x = r.timestamp, text = str(r.CuStepNo))
fig_days.update_layout(yaxis_range = [-0.2, 0.1])

fig_days.show()

In [ ]:
import matplotlib.pyplot as plt

for x in a.iloc[:, 3:32].columns:
    plt.figure(figsize=(15, 5), dpi=80)
    plt.plot(a["timestamp"], a[x])
    plt.title(x)

## Dynamic Time Warping

In [ ]:
a_dtw = a.drop(["timestamp", "day", "hour", "second", "dstep_p", "dstep_n",
               "timestamp_difference"], axis = 1)

In [ ]:
from dtw import dtw
from tslearn.metrics import dtw_path, dtw

# dtw_df soll die neuen Zeitreihen beinhalten
dtw_df = pd.DataFrame(columns=a_dtw.columns)

dtw_series_list = []

# Gehe über jedes Feature
for col in a_dtw.columns:
    print(col)

    dtw_series = []

    for x in a_dtw.CuStepNo.unique():
        # Datensatz mit nur diesem Schritt
        step_df = a_dtw[a_dtw.CuStepNo == x]

        # Längste Schrittdauer ermitteln (Achtung, sehr ausreißerempfindlich)
        # evtl. mean + 2 * std statt max?
       # max_length = int(a_dtw[a_dtw.CuStepNo == x].step_length.max())
        max_length = int(a_dtw[a_dtw.CuStepNo == x].step_length.mean() + 1.96 * a_dtw[a_dtw.CuStepNo == x].step_length.std())
        
        for b in a_dtw.batchn.unique():
            
            time_series = step_df[step_df.batchn == b][col].to_numpy()

            # DTW, um alte Zeitreihe an die neue Größe anzupassen (Berechnen der Distanzmatrizen)
            path, dist = dtw_path(time_series, np.zeros(max_length), 
                                  global_constraint="itakura", 
                                  itakura_max_slope=10)

            # Neue Zeitreihe, die dann in dtw_df übernommen wird
            new_time_series = np.zeros(max_length)

            # Ersetze die Nullen durch die Werte aus den alten Zeitreihen (beide haben die gleiche Länge)
            for i, j in path:
                new_time_series[j] = time_series[i]

            # Werte an dtw_series ranhängen
            dtw_series.append(new_time_series)
    
    # enthält alle bisher aufgenommenen Daten, könnte für einen effizienteren Algorithmus angepasst werden
    dtw_series_list.append(dtw_series)

# Für jedes Feature und jeden Schritt werden die Werte aus dtw_series zu dtw_df hinzugefügt, and add them to the new DataFrame
for i, col in enumerate(a_dtw.columns):
    dtw_df[col] = pd.concat([pd.Series(batch) for batch in dtw_series_list[i]])

In [ ]:
# Zeitunterschied zwischen den interpolierten Zeilen

dtw_df["timestamp_difference"] = 1

# Erstelle die Werte für step_length wie zuvor beim DataFrame a

for x in range(1, dtw_df.batchn.nunique() + 1):
    for i in dtw_df[dtw_df.batchn == x].CuStepNo.unique():
        dtw_df.loc[(dtw_df.batchn == x) & (dtw_df.CuStepNo == i), "step_length"] = dtw_df.loc[(dtw_df.batchn == x) & (dtw_df.CuStepNo == i), :].timestamp_difference.cumsum()
        
# Spalte kann wieder entfernt werden, da nicht mehr benötigt
dtw_df.drop("timestamp_difference", axis = 1, inplace = True)

In [ ]:
px.line(a[(a.DeviationID == 1) & (a.CuStepNo == 1)],  x = "step_length", 
                      y = "LevelMainTank", color = "batchn",
                      title = f"Datensatz vor DTW - DeviationID = {1}, CuStepNo: {1}",
                 ).update_layout(xaxis_range = [0, 323])

In [ ]:
px.line(dtw_df[(dtw_df.DeviationID == 1) & (dtw_df.CuStepNo == 1)],  x = "step_length", 
                      y = "LevelMainTank", color = "batchn",
                      title = f"Datensatz nach DTW - DeviationID = {1}, CuStepNo: {1}",
                 ).update_layout(xaxis_range = [0, 323])

In [ ]:
px.line(a[(a.batchn == 23) & (a.DeviationID == 1) & (a.CuStepNo == 1)],  x = "step_length", 
                      y = "LevelMainTank", 
                      title = f"Originaler Datensatz - DeviationID = {1}, batchn: {23}, CuStepNo: {1}"
                 ).update_layout(xaxis_range = [0, 323])                 

In [ ]:
px.line(dtw_df[(dtw_df.batchn == 23) & (dtw_df.DeviationID == 1) & (dtw_df.CuStepNo == 1)],  x = "step_length", 
                      y = "LevelMainTank", 
                      title = f"Datensatz nach DTW - DeviationID = {1}, batchn: {22}, CuStepNo: {1}"
                 )

## Machine Learning

In [ ]:
dtw_df.columns

Besser auch batchn entfernen

In [ ]:
dtw_df.drop("batch_duration", axis = 1, inplace = True)

In [ ]:
# Wir erstellen Trainings- und Testdaten in einem Verhältnis von 70:30. 
# Da Batches mit verschiedenen DevIDs chronologisch durchgegangen werden (bspw. kommen Batches mit DevID == 8 und == 10
# vermehrt in den letzten Tagen des erfassten Zeitraums), sammeln wir pro DeviationID 70% der Batches. 
# D.h. aus Batches mit DevID == 1 werden 70% dem Trainingsdatensatz hinzugefügt, die restlichen 30% dem Testdatensatz usw.

# train_test_split würde zu einer willkürlichen Aufteilung führen, die das Prinzip der Unabhängigkeit zwischen den
# Datenpunkten verletzen würden. Wir wollen die einzelnen Batches aufteilen.

train = pd.DataFrame()
test = pd.DataFrame()

for dev_id in dtw_df.DeviationID.unique():
    #print(dev_id)
    batches_with_devid = dtw_df[dtw_df.DeviationID == dev_id].batchn.unique()
    num_batches = len(batches_with_devid)
    train_batches = int(round(num_batches * 0.7))
    last_index_train = batches_with_devid[train_batches - 1]
    
    for b in batches_with_devid:
        if b <= last_index_train:
            train = train.append(dtw_df[(dtw_df.DeviationID == dev_id) & (dtw_df.batchn == b)])
        else:
            test = test.append(dtw_df[(dtw_df.DeviationID == dev_id) & (dtw_df.batchn == b)])

In [ ]:
# Speichere die Trainings- und Testdaten als Backup

backup_train = train.copy()
backup_test = test.copy()

In [ ]:
y_train = train.pop("DeviationID")
y_test = test.pop("DeviationID")

X_train = train
X_test = test

In [ ]:
import numpy as np
from scipy.spatial import distance
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# ist kommentiert, da das Modell zu lange läuft (zuletzt am 03.04., vor DTW, ausprobiert)

"""#custom metric
def DTW(a, b):   
    an = a.size
    bn = b.size
    pointwise_distance = distance.cdist(a.reshape(-1,1),b.reshape(-1,1))
    cumdist = np.matrix(np.ones((an+1,bn+1)) * np.inf)
    cumdist[0,0] = 0

    for ai in range(an):
        for bi in range(bn):
            minimum_cost = np.min([cumdist[ai, bi+1],
                                   cumdist[ai+1, bi],
                                   cumdist[ai, bi]])
            cumdist[ai+1, bi+1] = pointwise_distance[ai,bi] + minimum_cost

    return cumdist[an, bn]

#train
parameters = {'n_neighbors':[10]}
clf = GridSearchCV(KNeighborsClassifier(metric=DTW), parameters, cv=2, verbose=1)
clf.fit(X_train, y_train)



#evaluate
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))"""

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

X_train_sc = sc.fit_transform(X_train)
X_test_sc = sc.transform(X_test)

In [ ]:
# Trainiere einen Autoencoder auf skalierten Daten
# Über eine Encoderschicht mit ReLu-Aktivierungsfunktion und mit 2 Neuronen sollen die Daten rekonstruiert werden
# Weichen die rekonstruierten Daten zu stark von den originalen Inputdaten ab, meldet der Autoencoder eine Anomalie

from keras.optimizers import Adam, SGD
from keras.models import Model
from keras.layers import Input, Dense
from keras.callbacks import EarlyStopping

import torch

X_train_tensor =  np.asarray(X_train).astype('float32')
X_test_tensor =  np.asarray(X_test).astype('float32')
y_train_tensor =  np.asarray(y_train).astype('float32')
y_test_tensor =  np.asarray(y_test).astype('float32')


input_layer = Input(shape = (X_train.shape[1], ))
encoder_layer = Dense(10, activation = "relu")(input_layer)
decoder_layer = Dense(X_train.shape[1], activation = "linear")(encoder_layer)
        
autoencoder = Model(inputs = input_layer, outputs = decoder_layer)
        

        
autoencoder.compile(optimizer = SGD(learning_rate=0.01), loss = "mse")

early_stopping = EarlyStopping(monitor='val_loss', patience=10)
autoencoder.fit(X_train_tensor, y_train_tensor,
                epochs=5,
                batch_size=32,
                shuffle=True,
                validation_data=(X_test_tensor, y_test_tensor),
                callbacks=[early_stopping])

# use autoencoder to detect anomalies
threshold = np.mean(autoencoder.predict(X_train_tensor) - X_train_tensor) + 2 * np.std(autoencoder.predict(X_train_tensor) - X_train_tensor)
anomalies = np.where(autoencoder.predict(X_test_tensor) - X_test_tensor > threshold, 1, 0)


In [ ]:
sums = np.sum(anomalies, axis = 1)
print(f"Im Testdatensatz werden an {len(sums[sums > 0])} Datenpunkten Anomalien gemeldet.")

In [ ]:
print(f"Der Testdatensatz besteht aus {y_test.shape[0]} Datenpunkten, wobei {y_test[y_test == 1].shape[0]} Datenpunkte eine DeviationID > 1 aufweisen")

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
